# Projet Representation Learning for NLP

M2 MIASHS - Université de Lyon - Janvier 2026

Luan DECHERY – Mathis DERENNE – Enrique NASSEH FREITAS

---

# Introduction

Depuis la publication de l'article *"Attention Is All You Need"* (Vaswani et al., 2017), le paradigme du traitement automatique du langage (TAL/NLP) a profondément changé, marquant le début de l'ère des LLMs. Les approches précédentes, comme Bag-of-Words ou TF-IDF, représentaient les mots par des comptages de fréquences. D'autres approches comme Word2Vec créaient des embeddings capturant certaines relations sémantiques entre les mots. Cependant, ces modèles ne prenaient pas en compte un aspect tout aussi important du langage qui est le **contexte**.

En effet, les premiers embeddings ne prenaient pas en compte des informations comme la **position des mots** ("Pierre mange une pizza" diffère de "Une pizza mange Pierre") ou la **polysémie** (le mot "modèle" désigne un concept mathématique ou une profession selon le contexte). Pour résoudre cela, Vaswani et al. ont proposé l'architecture **Transformer**, reposant sur un mécanisme de **self-attention** qui permet de capturer les dépendances entre tous les mots d'une séquence. Contrairement aux approches séquentielles (RNN, GRU, LSTM), cette nouvelle architecture permet une parallélisation efficace des calculs sur GPU, réduisant significativement le temps d'entraînement.

Le Transformer est composé de deux parties principales : un **encodeur**, qui transforme les mots ou phrases en vecteurs contenant leur position et contexte, et un **décodeur**, qui génère du texte à partir de ces représentations via un mécanisme de cross-attention. Des architectures dérivées se sont ensuite spécialisées : **BERT** (Devlin et al., 2019) utilise une architecture *encoder-only* avec une attention bidirectionnelle pour des tâches de compréhension comme la classification, que nous allons réaliser dans ce projet. **GPT** (Radford et al., 2018) utilise une architecture *decoder-only* avec du masked self-attention pour des tâches de génération (agents conversationnels comme ChatGPT, Claude, etc.).

Ces modèles sont souvent appelés **LLM** (Large Language Models), notamment parce qu'ils sont entraînés sur d'immenses quantités de données textuelles. Leur entraînement se déroule généralement en deux phases. La première, appelée **pré-entraînement** (*pretraining*), permet au modèle d'apprendre le langage, la grammaire et le contexte de manière auto-supervisée ; elle aboutit à ce qu'on appelle un **modèle de base** (*foundation model*). La seconde phase, appelée **post-entraînement** ou **fine-tuning**, consiste à ajuster les poids du modèle pour l'adapter à une tâche spécifique. Parmi les méthodes récentes et accessibles pour cette étape, **LoRA** (Hu et al., 2021) permet un ajustement efficace en ne modifiant qu'une fraction des paramètres (que nous verrons plus en détail dans les parties dédiées).

Dans ce projet, nous abordons la tâche d'**inférence en langage naturel (NLI)** en français, qui consiste à déterminer la relation sémantique entre deux phrases : neutralité, contradiction ou conséquence (*entailment*).

| Relation | Description | Exemple |
|----------|-------------|---------|
| **Entailment** (Conséquence) | L'hypothèse découle logiquement de la prémisse | P: "Le chat dort sur le canapé" → H: "Un animal est sur un meuble" |
| **Contradiction** | L'hypothèse contredit la prémisse | P: "Le chat dort sur le canapé" → H: "Le chat court dehors" |
| **Neutral** | Aucune relation logique claire | P: "Le chat dort sur le canapé" → H: "Le chat aime le poisson" |

Le jeu de données est divisé en un ensemble d'entraînement contenant 5 010 exemples et un ensemble d'évaluation avec 2 490 exemples, équilibrés entre les trois classes. Cette tâche est fondamentale en NLP car elle évalue la capacité d'un modèle à raisonner sur le sens des phrases.

Ce projet implémente et compare plusieurs approches pour réaliser cette tâche :

- **Apprentissage en contexte** avec un LLM pré-entraîné (`02-in-context-learning.ipynb`)
- **Fine-tuning d'un encodeur** (tâche de classification multi-classes) (`03-encoder-tuning.ipynb`)
- **Fine-tuning d'un décodeur** avec LoRA (`04-decoder-tuning.ipynb`)

### Apprentissage en contexte

L'apprentissage en contexte consiste à interroger directement un modèle de langage pré-entrainé. Cette approche est la plus simple et n'ajuste pas les poids du modèle. On utilise comme modèle de base **Llama 3.2 3B Instruct**.

Plusieurs approches sont testés et comparés :
- zero-shot: le modèle doit répondre sans exemple préalable
- few-shot: le modèle reçoit quelques exemples annotés en contexte avant de répondre
- chaîne de raisonnement (Chain Of Thought (CoT)): le modèle est encouragé à expliquer son raisonnement avant de donner une réponse finale.

### Fine-tuning d'un encodeur

L'utilisation d'une architecture de type **encodeur** est particulièrement adaptée aux tâches de classification de séquences comme la NLI. Contrairement aux modèles génératifs, l'encodeur traite la séquence de manière bidirectionnelle, permettant à chaque mot d'intégrer l'information de l'ensemble de la prémisse et de l'hypothèse dès les premières couches du modèle.

Pour ce projet, nous nous appuyons sur **CamemBERT 2.0** et **CamemBERTa 2.0**, des évolutions de l'état de l'art pour la langue française.

* **Mécanisme de Classification :** Le modèle traite la paire de phrases comme une entrée unique. La représentation vectorielle du jeton spécial `[CLS]` (ou `<s>`), extraite de la dernière couche cachée, sert de résumé sémantique de la relation entre les deux phrases. Cette représentation est ensuite envoyée vers une tête de classification (couche dense) qui projette le vecteur vers les trois classes cibles : *entailment*, *neutral*, et *contradiction*.

* **Optimisation par LoRA (Low-Rank Adaptation) :** Afin de limiter le coût computationnel et de prévenir l'oubli catastrophique, nous utilisons la méthode **LoRA**. Au lieu de mettre à jour la totalité des paramètres du modèle, nous injectons des matrices de bas rang dans les couches d'attention (spécifiquement sur les modules *query* et *value*). Mathématiquement, pour une matrice de poids initiale $W_0 \in \mathbb{R}^{d \times k}$, la mise à jour est modélisée par :
$$W = W_0 + \Delta W = W_0 + BA$$
où $B \in \mathbb{R}^{d \times r}$ et $A \in \mathbb{R}^{r \times k}$ sont des matrices de rang $r \ll d$.

* **Entraînement :** Le processus repose sur un apprentissage supervisé classique avec une fonction de perte de type **Cross-Entropy**, ajustant uniquement les adaptateurs LoRA et la tête de classification.

### Fine-tuning d'un LLM

Le fine-tuning d'un modèle de type **décodeur** (LLM) transforme radicalement la nature de la tâche. Ici, l'inférence n'est plus traitée comme un problème de classification pur, mais comme une tâche de **génération de texte sous contrainte**. Nous utilisons pour cela la famille de modèles **Llama 3.2 1B**.

Cette approche se distingue par les étapes suivantes :

* **Formatage des données (Prompting) :** Chaque exemple est converti en une instruction textuelle structurée. Le modèle doit apprendre à prédire le prochain jeton (token) qui correspond à la classe correcte.
> **Exemple de prompt :**
> "Détermine la relation logique entre les deux phrases suivantes. Réponds uniquement par : conséquence, neutralité ou contradiction.
> Prémisse : [Phrase A] Hypothèse : [Phrase B] Relation :"

* **Apprentissage par Causal Language Modeling (CLM) :** Le modèle est entraîné à maximiser la probabilité de générer l'étiquette correcte à la suite du prompt. Contrairement à l'encodeur, le calcul de la perte se fait sur la partie "réponse" de la séquence générée.
* **Adaptation avec LoRA :** À l'instar de l'encodeur, LoRA est appliqué pour permettre un entraînement efficace sur des GPUs grand public. L'ajustement porte sur les poids du mécanisme d'auto-attention afin d'aligner la génération du modèle sur le vocabulaire spécifique des labels de notre dataset.
* **Inférence et Post-traitement :** Lors de l'évaluation, nous demandons au modèle de générer un jeton. Un mécanisme de mapping est ensuite appliqué pour convertir la chaîne de caractères générée (ex: "conséquence") en label numérique afin de calculer les métriques de performance (Accuracy, F1-score).

### Comparaison Encodeur vs Décodeur

L'architecture Transformer repose sur un mécanisme d'attention qui calcule les relations entre tous les mots d'une séquence :

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

où $Q$, $K$ et $V$ sont des projections linéaires de l'entrée. La différence entre encodeur et décodeur réside dans les positions que chaque mot peut observer.

L'**encodeur** utilise une attention bidirectionnelle : chaque mot accède à tous les autres mots de la séquence. Cette architecture est adaptée aux tâches de compréhension (classification, NLI). Le pré-entraînement s'effectue par *Masked Language Modeling* où le modèle prédit des mots masqués. BERT et CamemBERT illustrent cette approche.

Le **décodeur** utilise une attention causale : chaque mot ne voit que les mots précédents. Cette contrainte simule la génération de texte. Le pré-entraînement repose sur la prédiction du mot suivant. GPT et Llama adoptent cette architecture, mais peuvent être adaptés à la classification via des prompts.

| | **Encodeur** | **Décodeur** |
|---|---|---|
| Attention | Bidirectionnelle | Causale |
| Pré-entraînement | Masked LM | Next Token Prediction |
| Tâches | Classification, NLI | Génération, Chat |
| Exemples | BERT, CamemBERT | GPT, Llama |

### Méthode LoRA

Le fine-tuning classique modifie tous les paramètres du modèle, ce qui pose des problèmes de mémoire et de temps pour un modèle d'1 milliard de paramètres. **LoRA** (Low-Rank Adaptation) résout ce problème en ajoutant une modification de rang faible :

$$h = Wx + BAx$$

où $W$ est la matrice originale (gelée), $B \in \mathbb{R}^{d \times r}$ et $A \in \mathbb{R}^{r \times d}$ avec $r \ll d$. Seules $A$ et $B$ sont entraînées.

Pour une matrice $4096 \times 4096$ avec $r = 16$ : le fine-tuning classique entraîne 16.7M paramètres, LoRA n'en entraîne que 131K, soit une **réduction de 99.2%**.

| **Paramètre** | **Valeur** | **Rôle** |
|---|---|---|
| $r$ | 16 | Rang des matrices A et B |
| $\alpha$ | 32 | Facteur de mise à l'échelle |
| Modules | q, k, v, o_proj | Projections d'attention |
| Dropout | 0.05 | Régularisation |